In [ ]:
!pip install qiskit qiskit-aer qiskit-ibm-runtime matplotlib pandas numpy


In [ ]:
!pip install "qiskit[visualization]" matplotlib pillow pylatexenc




In [ ]:
# CELL 1: IMPORTS + FRONTEND (creates original_circuit_complex.png)
# Function: Builds "high-level source program" - 5-qubit entangled circuit with random rotations
# Shows: What goes INTO the quantum compiler [web:24]

from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Build complex test circuit (GHZ + rotations)
n_qubits = 5
qc_original = QuantumCircuit(n_qubits)
qc_original.h(range(n_qubits))  # Create superposition
for i in range(n_qubits-1):     # Forward CX chain (entanglement)
    qc_original.cx(i, i+1)
for i in range(n_qubits):       # Random 1-qubit rotations
    qc_original.rx(0.3 * (i+1), i)
    qc_original.ry(0.2 * (i+1), i)
for i in range(n_qubits-1):     # Backward CX chain
    qc_original.cx(i+1, i)
qc_original.measure_all()

print("=== ORIGINAL CIRCUIT (Source Program) ===")
print(f"Depth: {qc_original.depth()}, Size: {qc_original.size()}, CX gates: {qc_original.count_ops().get('cx', 0)}")

# Visualize input to compiler
fig0 = qc_original.draw('mpl')
fig0.savefig('original_circuit_complex.png', dpi=300, bbox_inches='tight')
plt.show()


=== ORIGINAL CIRCUIT (Source Program) ===
Depth: 10, Size: 28, CX gates: 8


In [ ]:
# CELL 2: TARGET HARDWARE MODEL
# Function: Defines FakeSherbrooke (127-qubit IBM backend with connectivity + noise)
# Shows: Compiler's "target architecture" constraints [web:6][web:9]

from qiskit_ibm_runtime.fake_provider import FakeSherbrooke
from qiskit_aer import AerSimulator

backend = FakeSherbrooke()  # Real IBM hardware model (not ideal simulator)
sim = AerSimulator()        # For execution comparison

print("=== TARGET HARDWARE ===")
print(f"Backend: {backend.name}")
print(f"Qubits: {backend.num_qubits}")
print(f"Basis gates: {backend.operation_names}")

# FIX: use get_edges() to access the coupling map as a list of qubit pairs
print(f"Coupling map sample: {backend.coupling_map.get_edges()[:5]}...")  # Shows qubit connectivity


=== TARGET HARDWARE ===
Backend: fake_sherbrooke
Qubits: 127
Basis gates: ['measure', 'rz', 'if_else', 'switch_case', 'reset', 'delay', 'sx', 'for_loop', 'id', 'x', 'ecr']
Coupling map sample: [(1, 0), (1, 2), (3, 2), (4, 3), (4, 15)]...


In [ ]:
# CELL 3: COMPILER PASSES - LEVEL 0 (no optimization)
# Function: Baseline transpilation (layout + routing only)
# Shows: How compiler maps to hardware without optimization [web:22]

pm0 = generate_preset_pass_manager(optimization_level=0, backend=backend, seed_transpiler=12345)
qc_level0 = pm0.run(qc_original)

print("\n=== LEVEL 0 (Layout + Routing Only) ===")
print(f"Depth: {qc_level0.depth()}, CX: {qc_level0.count_ops().get('cx', 0)}, Size: {qc_level0.size()}")
print(f"Qubit mapping: {qc_level0.layout}")
print(f"Basis gates used: {list(qc_level0.count_ops().keys())}")


In [ ]:
# CELL 4: COMPILER PASSES - LEVELS 1-3 + VISUALS (creates optimized_circuit_complex.png)
# Function: Full optimization pipeline, pass manager visualization
# Shows: How higher levels reduce depth/CX gates [web:1][web:8]

results = []
optimized_circuits = {}

for level in [1, 2, 3]:
    pm = generate_preset_pass_manager(optimization_level=level, backend=backend, seed_transpiler=12345)
    qc_opt = pm.run(qc_original)
    optimized_circuits[level] = qc_opt

    depth = qc_opt.depth()
    cx_count = qc_opt.count_ops().get('cx', 0)
    size = qc_opt.size()

    results.append({'Level': level, 'Depth': depth, 'CX_Gates': cx_count, 'Size': size})
    print(f"Level {level}: Depth={depth}, CX={cx_count}, Size={size}")

    # Show Level 2 circuit transformation
    if level == 2:
        fig1 = qc_opt.draw('mpl')
        fig1.savefig('optimized_circuit_complex.png', dpi=300, bbox_inches='tight')
        plt.show()

        # Show pass manager pipeline (compiler stages)
        print("\nLevel 2 Pass Manager Stages:")
        print(pm.draw())  # Visual pipeline like classical compiler


In [ ]:
# CELL 5: METRICS COMPARISON (creates metrics_comparison_complex.png)
# Function: Quantifies compiler effectiveness across optimization levels
# Shows: Depth/CX reduction = better hardware execution [web:23]

# Add level 0 to results
results.insert(0, {'Level': 0, 'Depth': qc_level0.depth(), 'CX_Gates': qc_level0.count_ops().get('cx', 0), 'Size': qc_level0.size()})

df_results = pd.DataFrame(results)
print("\n=== COMPILER OPTIMIZATION RESULTS ===")
print(df_results)

# Bar charts (like compiler benchmark)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
df_results.plot(x='Level', y='Depth', kind='bar', ax=ax1, title='Circuit Depth Reduction')
df_results.plot(x='Level', y='CX_Gates', kind='bar', ax=ax2, title='Two-Qubit Gate Count')
plt.tight_layout()
plt.savefig('metrics_comparison_complex.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# CELL 6: EXECUTION + VERIFICATION (creates results_histogram_complex.png)
# Function: Proves compiler preserves quantum semantics
# Shows: Different circuits → same output distribution [web:22]

# Run original vs optimized
job_orig = sim.run(qc_original, shots=1024)
job_opt = sim.run(optimized_circuits[2], shots=1024)

counts_orig = job_orig.result().get_counts()
counts_opt = job_opt.result().get_counts()

print("\n=== SEMANTIC EQUIVALENCE ===")
print("Original top-3:", sorted(counts_orig.items(), key=lambda x: x[1], reverse=True)[:3])
print("Level-2 top-3:", sorted(counts_opt.items(), key=lambda x: x[1], reverse=True)[:3])

# Histograms prove equivalence
fig2, (ax3, ax4) = plt.subplots(1, 2, figsize=(12, 4))
plot_histogram(counts_orig, ax=ax3, title='Original Circuit')
plot_histogram(counts_opt, ax=ax4, title='Optimized Level 2')
plt.savefig('results_histogram_complex.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Quantum Compiler Demo Complete!")
print("Files generated: original_circuit_complex.png, optimized_circuit_complex.png, metrics_comparison_complex.png, results_histogram_complex.png")
